In [1]:
import time
import crcmod
import threading

In [2]:
def int_to_binary(number, length):
    binary = format(number, '0' + str(length) + 'b')
    return [int(k) for k in list(binary)]

def int_list_to_string(int_list):
    return ''.join([str(k) for k in int_list])

def int_list_to_int(int_list):
    return int(int_list_to_string(int_list), 2)

def string_data_to_int_list(string_data):
    res = ''.join(format(i, '08b') for i in bytearray(string_data, encoding ='utf-8'))
    return [int(k) for k in list(res)]

def int_list_to_string_data(int_list):
    return bytes([int(int_list_to_string(int_list)[i:i+8], 2) for i in range(0, len(int_list), 8)]).decode('utf-8')

def clear_output_files(num_nodes):
    # Clear all output files
    for i in range(num_nodes):
        f = open(f"out_{i}.txt", "w")
        f.write("")
        f.close()

In [3]:
class FlexRayDataFrame:
    def __init__(self, 
                 payload_preamble, 
                 frame_type, 
                 frame_id, 
                 cycle_count, 
                 data_bits
                 ):
        # Experiment with different CRC polynomials
        self.header_crc_func = crcmod.predefined.Crc('crc-8')
        self.trailer_crc_func = crcmod.predefined.Crc('crc-24-flexray-a')
        self.header = self.generate_header_frame(payload_preamble, 
                                                 frame_type, 
                                                 frame_id, 
                                                 cycle_count, 
                                                 1 + int(len(data_bits) / 8))
        self.payload = data_bits
        self.trailer = self.generate_trailer_bits(data_bits)

    def generate_header_frame(self, 
                              payload_preamble, 
                              frame_type, 
                              frame_id, 
                              cycle_count, 
                              len_data_bits
                              ):
        header = [0] * 40
        header[1] = payload_preamble
        if frame_type == "NULL":
            header[2] = 1
        elif frame_type == "SYNC":
            header[3] = 1
        elif frame_type == "STARTUP":
            header[4] = 1
        header[5 : 16] = int_to_binary(frame_id, 11)
        header[16 : 23] = int_to_binary(len_data_bits, 7)
        # Bits 24 to 26 are 0's as CRC can be computed for polynomial of power 2
        crc = self.header_crc_func.new(int_list_to_string(header[1 : 23]).encode('utf-8')).crcValue
        header[26 : 34] = int_to_binary(crc, length = 8)
        header[-6 : ] = int_to_binary(cycle_count, 6)
        return header
    
    def generate_trailer_bits(self, data_bits):
        trailer_crc = self.trailer_crc_func.new(int_list_to_string(data_bits).encode('utf-8')).crcValue
        return int_to_binary(trailer_crc, length = 24)

In [4]:
class GlobalCounter:
    def __init__(self, 
                 max_value, 
                 delay_ms, 
                 num_nodes
                 ):
        self.max_value = max_value
        self.current_value = 0
        self.comm_cycle = -1
        self.delay_s = delay_ms / 1000
        self.clock_thread = threading.Thread(target = self._run)
        self.stop_event = False
        self.num_nodes = num_nodes

    def start(self):
        self.comm_cycle = 0
        self.stop_event = False
        self.clock_thread = threading.Thread(target = self._run)
        self.clock_thread.start()

    def _run(self):
        last_save_time = time.time()
        while not self.stop_event:
            x = time.time()
            if x - last_save_time > self.delay_s:
                last_save_time = x
                self.current_value = (self.current_value + 1) % self.max_value
                
                if self.current_value == 0:
                    self.comm_cycle = (self.comm_cycle + 1) % 64
                # print("Clock: ", self.current_value)

    def stop(self):
        self.stop_event = True
        self.clock_thread.join()
    
    def modify_value(self, new_value):
        self.current_value = new_value

class DataBus:
    def __init__(self, 
                 num_nodes, 
                 max_clock_val = 1024, 
                 delay_ms = 10
                 ):
        self.max_clock_val = max_clock_val
        null_frame = FlexRayDataFrame(1, "NULL", 0, 0, [0])
        self.init_ports = [[null_frame] * (max_clock_val - 1) for _ in range(num_nodes)]
        self.data_bus = self.init_ports
        self.clock = GlobalCounter(max_clock_val, delay_ms, num_nodes)
        self.num_nodes = num_nodes
        self.data_cleaning_thread = threading.Thread(target = self.data_cleaning)
        self.stop_data_cleaning_event = False

    def start_clock(self):
        self.clock.start()
    
    def stop_clock(self):
        self.clock.stop()

    def start_data_cleaning(self):
        self.stop_data_cleaning_event = False
        self.data_cleaning_thread = threading.Thread(target = self.data_cleaning)
        self.data_cleaning_thread.start()

    def stop_data_cleaning(self):
        self.stop_data_cleaning_event = True
        self.data_cleaning_thread.join()

    def data_cleaning(self):
        # Clearing the data in the queue when the clock reaches 0
        while not self.stop_data_cleaning_event:
            if self.clock.current_value == 0:
                self.data_bus = self.init_ports

    def initialize(self):
        self.start_clock()
        self.start_data_cleaning()
        
    def send_data(self,
                  slot_num, 
                  node_id, 
                  flexray_frame
                  ):
        
        min_val = 1 + int(node_id * self.max_clock_val / self.num_nodes)
        max_val = 1 + int((node_id + 1) * self.max_clock_val / self.num_nodes)

        # If the node is not allowed in the slot
        if slot_num < min_val and slot_num >= max_val:
            print("Node not allowed to send in the slot")
            return False
        
        for i in range(self.num_nodes):
            self.data_bus[i][slot_num - 1] = flexray_frame
        return self.clock.current_value

In [5]:
num_nodes = 4
max_clock_val = 41
delay_ms = 500
data_bus = DataBus(num_nodes, max_clock_val = max_clock_val, delay_ms = delay_ms)

In [6]:
class FlexRayNode:
    def __init__(self, node_id, num_nodes = 4, max_clock_val = 1024):
        self.Node_ID = node_id
        self.data_min_clk = 1 + int(node_id * (max_clock_val - 1) / num_nodes)
        self.data_max_clk = 1 + int((node_id + 1) * (max_clock_val - 1) / num_nodes)
        self.read_mode = False
        self.send_null_frame_flag = False
        self.stop_update_read_mode = False
        self.stop_read_data_event = False
        self.stop_send_null_frame_event = False
        self.update_read_mode_thread = threading.Thread(target = self.update_read_mode)
        self.read_data_thread = threading.Thread(target = self.read_data_from_bus)
        self.send_null_frame_thread = threading.Thread(target = self.send_null_frame)

    def parse_frame(self, frame):
        parsed_frame = {}
        
        header = frame.header
        parsed_frame["payload_preamble"] = header[1]
        parsed_frame["frame_type"] = "DATA"
        if header[2] == 1:
            parsed_frame["frame_type"] = "NULL"
        elif header[3] == 1:
            parsed_frame["frame_type"] = "SYNC"
        elif header[4] == 1:
            parsed_frame["frame_type"] = "STARTUP"
        parsed_frame["frame_id"] = int_list_to_int(header[5 : 16])
        parsed_frame["data_length"] = int_list_to_int(header[16 : 23])
        parsed_frame["crc"] = int_list_to_int(header[26 : 34])
        parsed_frame["cycle_count"] = int_list_to_int(header[-6 : ])

        parsed_frame["data"] = frame.payload

        parsed_frame["trailer_crc"] = frame.trailer

        return parsed_frame
    
    def parse_string_data(self, string_data):
        return int_list_to_string_data(string_data)

    def read_data_from_bus(self):
        prev_clk = 0
        while not self.stop_read_data_event:
            # Read only if the clock value is not in the range of write mode of node
            if not self.read_mode:
                continue
            clk_val = data_bus.clock.current_value
            if clk_val == prev_clk:
                continue
            prev_clk = clk_val
            
            flexray_frame = data_bus.data_bus[self.Node_ID][clk_val - 1]
            parsed_frame = self.parse_frame(flexray_frame)
            
            f = open(f"out_{self.Node_ID}.txt", "a")
            # If NULL frame received
            if parsed_frame["frame_type"] == "NULL":
                # f.write(f"{clk_val}\n")
                pass

            # If SYNC frame received
            elif parsed_frame["frame_type"] == "SYNC":
                # print(f"Node {self.Node_ID} received SYNC frame")
                pass

            # If STARTUP frame received
            elif parsed_frame["frame_type"] == "STARTUP":
                # print(f"Node {self.Node_ID} received STARTUP frame")
                pass
            
            # If DATA frame received
            else:
                received_message = self.parse_string_data(parsed_frame["data"])
                f.write(f"Node {self.Node_ID} received data at clock {clk_val}\nMessage is:\n")
                f.write(received_message + "\n")
            f.close()

    def send_null_frame(self):
        prev_clk = 0
        null_frame = FlexRayDataFrame(0, "NULL", self.Node_ID, data_bus.clock.comm_cycle, [0])
        while not self.stop_send_null_frame_event:
            if self.send_null_frame_flag:
                clk = data_bus.clock.current_value
                if clk == prev_clk:
                    continue
                prev_clk = clk

                if not self.read_mode:
                    data_bus.send_data(clk, self.Node_ID, null_frame)

    def update_read_mode(self):
        prev_clk = 0
        while not self.stop_update_read_mode:
            clk = data_bus.clock.current_value
            if clk == prev_clk:
                continue
            prev_clk = clk
            # f = open(f"out_{self.Node_ID}.txt", "a")
            if clk >= self.data_min_clk and clk < self.data_max_clk:
                self.read_mode = False
                # f.write(f"Node {self.Node_ID} is in write mode at clock {clk}\n")
            else:
                self.read_mode = True
                # f.write(f"Node {self.Node_ID} is in read mode at clock {clk}\n")
            # f.close()

    def send_data_event(self, data_bits):
        send_data_thread = threading.Thread(target = self.send_data, args = (data_bits,))
        send_data_thread.start()
        send_data_thread.join()
        self.send_null_frame_flag = True

    def send_data(self, data):
        # Maximum size of data for a frame is 254 bytes
        max_data_size = 254 * 8
        len_data = len(data)
        num_frames = 1 + int(len_data / max_data_size)
        flexray_frames = []
        for i in range(num_frames):
            frame = FlexRayDataFrame(0, "DATA", self.Node_ID, data_bus.clock.comm_cycle, data[i * max_data_size : (i + 1) * max_data_size])
            flexray_frames.append(frame)

        self.send_null_frame_flag = False
        num_frames_sent = 0
        while num_frames_sent < num_frames:
            if self.read_mode:
                continue
            clk = data_bus.clock.current_value
            frame_to_send = flexray_frames[num_frames_sent]
            send_success = data_bus.send_data(clk + 1, self.Node_ID, frame_to_send)
            if not send_success:
                print("Data not sent")
                break
            num_frames_sent += 1

    def start_node(self):
        self.send_null_frame_flag = True
        self.stop_update_read_mode = False
        self.stop_read_data_event = False
        self.stop_send_null_frame_event = False
        self.update_read_mode_thread = threading.Thread(target = self.update_read_mode)
        self.read_data_thread = threading.Thread(target = self.read_data_from_bus)
        self.send_null_frame_thread = threading.Thread(target = self.send_null_frame)
        self.update_read_mode_thread.start()
        self.read_data_thread.start()
        self.send_null_frame_thread.start()

    def stop_node(self):
        self.send_null_frame_flag = False
        self.stop_update_read_mode = True
        self.stop_read_data_event = True
        self.stop_send_null_frame_event = True
        self.update_read_mode_thread.join()
        self.read_data_thread.join()
        self.send_null_frame_thread.join()

In [7]:
data_bus.initialize()
clear_output_files(num_nodes)

In [8]:
nodes = []
for i in range(num_nodes):
    node = FlexRayNode(node_id = i, num_nodes = num_nodes, max_clock_val = max_clock_val)
    nodes.append(node)
    nodes[-1].start_node()

In [9]:
message = f"Hello from node {nodes[3].Node_ID}!"
nodes[3].send_data_event(string_data_to_int_list(message))

In [10]:
message = f"Hello from node {nodes[2].Node_ID}!"
nodes[2].send_data_event(string_data_to_int_list(message))

In [11]:
message = f"Hello from node {nodes[1].Node_ID}!"
nodes[1].send_data_event(string_data_to_int_list(message))

In [10]:
# for node in nodes:
#     node.stop_node()